<a href="https://colab.research.google.com/github/voxfox01/2025-Harvard-NASA-Collaboration/blob/main/LLM_Classification_model_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
# Need Install libraries
!pip install s3fs --quiet
!pip install ctransformers --no-binary ctransformers --quiet
!pip install -U bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00


In [ ]:
# Data Manipulations
import pandas as pd
import numpy as np

# System
import os
import sys
import gc

# Deep learning
import torch

# LLM
#from ctransformers import AutoModelForCausalLM
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

# Model repositories
from huggingface_hub import notebook_login, login

# AWS
import s3fs

# Colab
from google.colab import userdata

## Credentials variables

In [ ]:
# AWS Environment variables
os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('nasa_aws_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('nasa_aws_secret_key')
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

In [ ]:
# Saving path
data_path = "/content/drive/MyDrive/Harvard/Capstone/Classification/"

# Load Data (GlOBE site data)

In [ ]:
globe_site_data = pd.read_csv(data_path + "globe_site_data_20250227.csv", dtype=str, low_memory=False)
globe_site_data.columns = globe_site_data.columns.str.strip()

print(globe_site_data.shape)
globe_site_data.head(3)

(28174, 25)


,site_id,name,latitude,longitude,elevation,elevation_type,location_source,comments,activated_at,water_body_name,...,area_unit,depth,depth_unit,water_sample_location,can_see_bottom,bank_material,bedrock_type,freshwater_habitat,salt_habitat,harvard_class
0,1,GLOBE Boulder,40.1,-105.17,1655,NaN,gps,NaN,13:18.1,Mosquito Source Water Body,...,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,Marsh
1,1000,Surface Water Site 01:SWS-01,6.4167,2.6667,16.1,ellipsoidal,gps,please replace with Hydrology Site Comments,4/4/2004 0:00,NaN,...,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,River_Stream
2,100001,22JGS446679,-25.586154,-48.564832,8.1,NaN,gps,NaN,24:31.2,Mosquito Habitat,...,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN,River_Stream


In [ ]:
# S3 data url
s3_url = "s3://hes-dscap-staging/globe_site_data/globe_siteinfo_20250227.csv"
globe_site_data = pd.read_csv(s3_url, dtype=str, low_memory=False)
globe_site_data.columns = globe_site_data.columns.str.strip() # Remove trailing whitespaces from column names

print(globe_site_data.shape)
globe_site_data.head(3)

(28174, 24)


,site_id,name,latitude,longitude,elevation,elevation_type,location_source,comments,activated_at,water_body_name,...,area,area_unit,depth,depth_unit,water_sample_location,can_see_bottom,bank_material,bedrock_type,freshwater_habitat,salt_habitat
0,1,GLOBE Boulder,40.1,-105.17,1655.0,NaN,gps,NaN,2018-09-12 23:13:18.149608,Mosquito Source Water Body,...,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN
1,1000,Surface Water Site 01:SWS-01,6.4167,2.6667,16.1,ellipsoidal,gps,please replace with Hydrology Site Comments,2004-04-04 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN
2,100001,22JGS446679,-25.586154,-48.564832,8.1,NaN,gps,NaN,2017-10-04 20:24:31.16618,Mosquito Habitat,...,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,NaN,NaN


In [ ]:
# S3 data url
s3_url = "s3://hes-dscap-staging/GLOBE_Data/transparencies_1995-03-26_2025-01-11.csv"
globe_df = pd.read_csv(s3_url, dtype=str, low_memory=False)
globe_df.columns = globe_df.columns.str.strip() # Remove trailing whitespaces from column names

print(globe_df.shape)
globe_df.head(3)

(160273, 22)


,countryCode,countryName,elevation,organizationId,organizationName,protocol,siteId,siteName,transparenciesComments,transparenciesMeasuredAt,...,transparenciesSensorTurbidityNtu,transparenciesTransparencyDiskDoesNotDisappear,transparenciesTransparencyDiskImageDisappearanceM,transparenciesTubeImageDisappearanceCm,transparenciesTubeImageDoesNotDisappear,transparenciesUserid,transparenciesWaterBodyState,latitude,longitude,transparenciesGlobeTeams
0,SAU,Saudi Arabia,27.0,6509520,Saudi Arabia GLOBE v-School,transparencies,100568,Ali Bin Abi Talib Secondary School,NaN,2025-01-11 09:02:00,...,NaN,false,NaN,60,false,13136805,normal,21.577892,39.202318,NaN
1,SAU,Saudi Arabia,18.0,6509520,Saudi Arabia GLOBE v-School,transparencies,369589,Prince Majed Park,NaN,2025-01-11 09:02:00,...,NaN,false,NaN,60,false,13136805,normal,21.590226,39.182741,NaN
2,SAU,Saudi Arabia,0.0,6509520,Saudi Arabia GLOBE v-School,transparencies,370172,Salman Bay Beach,NaN,2025-01-11 09:02:00,...,NaN,false,NaN,60,false,13136805,normal,21.859427,38.978633,NaN


- Example of looking at instance

# EDA

## Current types of water bodies


In [ ]:
globe_site_data.columns

Index(['site_id', 'name', 'latitude', 'longitude', 'elevation',
       'elevation_type', 'location_source', 'comments', 'activated_at',
       'water_body_name', 'water_body_type', 'water_body_source',
       'bank_to_bank_distance', 'bank_to_bank_distance_unit', 'area',
       'area_unit', 'depth', 'depth_unit', 'water_sample_location',
       'can_see_bottom', 'bank_material', 'bedrock_type', 'freshwater_habitat',
       'salt_habitat'],
      dtype='object')

In [ ]:
globe_site_data.water_body_name.unique().tolist()

['Mosquito Source Water Body',
 nan,
 'Mosquito Habitat',
 'traps',
 'river',
 'KAIKORAI  STREAM',
 'Marine, Shakespeare Bay',
 'Middle Patuxent River',
 'Unnamed stream, tributary to Opihi River',
 'River mouth',
 'بئر',
 'Sandstone Stream',
 'Sandstone Catchment',
 'sandstone cachtment',
 'Waters of Leith',
 'watres of leith',
 'waters of leith',
 'major tributary of the kaikorai stream',
 'Kaikorai Estuary',
 'Olšava',
 'خزان ارضي',
 'Río Huallaga',
 'Lake Rotorua',
 'Lake Okataina',
 'Hamurana',
 'lake',
 'Arroyo Canelón Chico',
 'Poormans Stream',
 'Wierzyca',
 'Próbka',
 'Poorman Valley Stream',
 'Waihopai River',
 'Oreti River (Esuary)',
 'Otepuni Stream',
 'Wyndham Wildlife Refuge Pond',
 'Unnamed Stream tributory  to Mangahou river',
 'Tomahawk Lagoon, Northern Arm',
 'Kwethluk River',
 'gohareya',
 'Waiaua River',
 'Unnamed Drainage Ditch',
 'Otama steam',
 'Otama Stream',
 'falaj',
 'Otama stream',
 'Taylor River, White Bridge',
 'Marlborough Sounds, Blumine Is Beach',
 'Wai

In [ ]:
globe_site_data.water_body_source.unique().tolist()

[nan,
 'other',
 'stream',
 'bay',
 'river',
 'estuary',
 'lake',
 'pond',
 'ocean',
 'reservoir',
 'ditch',
 'agriculture',
 'marsh_swamp',
 'puddles_animal']

In [ ]:
# In the globe_site_data dataframe get the distribution of types of water_body_source
globe_site_data.water_body_source.value_counts()

,count
water_body_source,
river,2899
stream,2334
pond,1366
other,917
lake,739
reservoir,449
ocean,361
bay,285
ditch,192


In [ ]:
# how many nan are in globe_site_data dataframe's water_body_source
globe_site_data.water_body_source.isna().sum()


np.int64(18162)

In [ ]:
# Create a pie chart that includes the nulls for globe_site_data.water_body_source
import matplotlib.pyplot as plt

# Create a pie chart that includes the nulls for globe_site_data.water_body_source
water_body_source_counts = globe_site_data.water_body_source.value_counts(normalize=True) # Removed the extra indent and assigned to a variable

- water_body_source

In [ ]:
globe_site_data.water_body_type.unique().tolist()

['unknown', 'fresh_water', 'salt_water', 'brackish', nan]

# LLM Selection

We selected Meta's Llama 3.1 8B model as the LLM. The model was conditioned to adopt the analytical reasoning of a professional oceanographer through prompt engineering. This enabled the model to interpret each of GLOBE site's geolocation, textual comments, and water body names with a contextual understanding informed by domain-specific heuristics.


In [ ]:
# Login creditials for huggingface.com
login(userdata.get('HF_TOKEN'))

## LLM configuration

In [ ]:
# LLM name
model_id = "meta-llama/Llama-3.1-8B-Instruct"

# Model config
config = AutoConfig.from_pretrained(model_id)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Add pad token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})

# Model weights
my_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                config=config,
                                                torch_dtype=torch.float16,
                                                #load_in_8bit=True,  # or load_in_4bit=True if hitting max memory
                                                device_map="auto")
my_model.resize_token_embeddings(len(tokenizer))



config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128257, 4096)

## LLM Prompt Template
The template comprised six sections:
1.     <b>Expert Persona Definition:</b> Instructed the model to adopt the perspective of an experienced oceanographer.
2.      <b>Objective Clarification:</b> Clearly stated the goal of classifying the water body type based on provided metadata.
3.      <b>Label Definitions:</b> Provided detailed descriptions and examples for each of the seven classification categories: River/Stream, Pond/Lake, Marsh, Ocean, Non-natural water source, Inconclusive, and Insufficient information.
4.      <b>Data Presentation:</b> Supplied relevant GLOBE data fields for each observation, including latitude, longitude, water body names, comments, and other pertinent metadata.
5.      <b>Contradiction Handling:</b> Included examples demonstrating how to manage conflicting or ambiguous data entries.
6.      <b>Response Formatting:</b> Specified the expected structure and key phrases for the model's output to facilitate downstream processing.


In [ ]:
def llm_labeler(description: str) -> str:

    prompt = f"""Assume the role of Dr. Ivona Cetinić, a distinguished biological oceanographer affiliated with NASA's Goddard Space Flight Center and the Universities Space Research Association.
    Dr. Cetinić specializes in ocean biogeochemistry and phytoplankton ecology, employing advanced optical tools to elucidate the role of phytoplankton diversity in oceanic biogeochemical cycles.
    She serves as the Science Lead for Ocean Biogeochemistry on NASA's PACE (Plankton, Aerosol, Cloud, ocean Ecosystem) mission and has been the Project Scientist for the EXPORTS (EXport Processes in the Ocean from RemoTe Sensing) campaign.

    1) River_Stream
    2) Pond_Lake
    3) Marsh
    4) Ocean
    5) Non-natural water resource
    6) Inconclusive
    7) Insufficient information

    Description of each classification:
    1) River_Steam: This classification is used when the water testing observation comes from running water from a river, creek, estuary, or stream flowing into a sea channel or lake.
    2) Pond_Lake: This classification is used when the water testing observation comes from still water formed naturally into a pond or a lake.
    3) Marsh: This classification is used when the water testing observation comes from shallow water areas dominated by plants often found along the edges of lakes, rivers, swamps, or coastlines.
    4) Ocean: This classification is used when the water testing observation comes from the ocean, sea, or bay connected to the ocean.
    5) Non-natural water source: This classification is used when the water testing observation comes from water sources that were significantly altered by human activity such as reservoirs, canals, or man-made ditches, channels to drain or irrigate land.
    6) Inconclusive: This classification is used when the water testing observation comes from a source of an object structure such as a container, pot, bowl, water tank, water well, or scarface of an object or plant.
    7) Insufficient information: This classification is used when there is not enough information to determine the class or information contradicts itself.


    Here is the observation infomation please classify given this information: {description}
    Use the Latitude, Longitude, Comments, Water body name, Water body type, Water body source to help you classify.
    Your answer should not contradict the given data elements. For example, if Water body type: is salt_water it can not be River_Stream due to the stream having salt content.
    Your answer must be in this format with nothing else included, Answer: classification label name """.strip()

    # Encode the prompt
    #encoded_input = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True)
    #encoded_input = encoded_input.to(my_model.device)
    encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    encoded_input = {key: val.to(my_model.device) for key, val in encoded_input.items()}

    # Model prediction
    with torch.no_grad():
        output_ids = my_model.generate(
            **encoded_input,
            max_new_tokens=20,
            #max_length=128,
            temperature=0.2,
            num_beams=1,       # No beam search for simplicity
            no_repeat_ngram_size=2,
            #pad_token_id=tokenizer.eos_token_id
            pad_token_id=tokenizer.pad_token_id
        )

    # Decode output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
    full_text = output_text
    answer_split = output_text.split("Answer")
    if len(answer_split) > 1:
        output_text = answer_split[-1].strip()
    else:
        output_text = 'inconclusive'

    # Classifier options
    river_options = ['river', 'stream', 'river_stream','creek', 'estuary','River_stream','River Steam','river stream','River_Stream','river/stream']
    pl_options = ['pond', 'lake', 'pond_lake','Pond/Lake','pond/lake','Pond_lake']
    marsh_options = ['marsh', 'swamp']
    ocean_options = ['ocean', 'sea', 'coastal', 'coast','bay']
    non_natural_options = ['non-natural','non-natural water source','reservoirs', 'canals','ditche', 'channel','Non_natural_water_source','non_natural_water_resource']
    inconclusive_options = ['inconclusive', 'container', 'bowl','plant', 'pot']
    insufficient_options = ['insufficient', 'information','insufficient information']


    for word in river_options:
        if word in output_text.lower():
            return 'River_Stream', output_text
    for word in pl_options:
        if word in output_text.lower():
            return 'Pond_Lake', output_text
    for word in marsh_options:
        if word in output_text.lower():
            return 'Marsh', output_text
    for word in ocean_options:
        if word in output_text.lower():
            return 'Ocean', output_text
    for word in non_natural_options:
        if word in output_text.lower():
            return 'Non-natural', output_text
    for word in inconclusive_options:
        if word in output_text.lower():
            return 'Inconclusive', output_text
    for word in insufficient_options:
        if word in output_text.lower():
            return 'Insufficient', output_text

    return 'Unknown', full_text

# Classication Loop
- Iterating over metadata records

In [ ]:
# drop havard_class column
#globe_site_data = globe_site_data.drop(columns=['harvard_class'])

In [ ]:
unknown_list = []

# Column names
column_names = [col.replace('_', ' ').capitalize() + ':' for col in globe_site_data.columns]

# Pickup where model left off
if 'harvard_class' not in globe_site_data.columns:
    globe_site_data['harvard_class'] = None

current_idx = globe_site_data.index[globe_site_data['harvard_class'].isna()].tolist()


# Apply classification
for count, idx in enumerate(current_idx, start=1):
    row = globe_site_data.loc[idx]

    # Observation location info as a string
    row_info = "\n".join(
        f"{col_name} {value}" for col_name, value in zip(column_names, row[globe_site_data.columns])
    )

    # LLM classification prediction
    llm_class_response, output_text = llm_labeler(row_info)
    if llm_class_response == 'Unknown':
        unknown_list.append(output_text)

    # Update dataframe
    globe_site_data.at[idx, 'harvard_class'] = llm_class_response


    # Checkpoint save in data_path
    if count % 100 == 0:
        globe_site_data.to_csv(data_path + "globe_site_data_20250227.csv", index=False)

        # Print the last 5 result from the unknown list
        new_five_exmaples = unknown_list[-5:]
        print(f"Processed {count} rows. Last 5 results:")
        for example in new_five_exmaples:
            print(example)
        print("\n\n\n")

# Final save once complete
globe_site_data.to_csv(data_path + "globe_site_data_20250227.csv", index=False)
print("Processing complete. Final DataFrame saved.")

Processed 100 rows. Last 5 results:
You are a data scientist and oceanographer at NASA reviewing turbidity observations.
    Classify the following observation into exactly one of these five categories of water bodies.
    1) River_Stream
    2) Pond_Lake
    3) Marsh
    4) Ocean
    5) Non-natural water resource
    6) Inconclusive
    7) Insufficient information

    Description of each classification:
    1) River_Steam: This classification is used when the water testing observation comes from running water from a river, creek, estuary, or stream flowing into a sea channel or lake.
    2) Pond_Lake: This classification is used when the water testing observation comes from still water formed naturally into a pond or a lake.
    3) Marsh: This classification is used when the water testing observation comes from shallow water areas dominated by plants often found along the edges of lakes, rivers, swamps, or coastlines.
    4) Ocean: This classification is used when the water testing o

# Additional iteration to classify Unknown values

In [ ]:

globe_site_data = pd.read_csv(data_path + "globe_site_data_20250227.csv", dtype=str, low_memory=False)
globe_site_data.columns = globe_site_data.columns.str.strip()

globe_site_data[globe_site_data['harvard_class'] == 'Unknown']

,site_id,name,latitude,longitude,elevation,elevation_type,location_source,comments,activated_at,water_body_name,...,area_unit,depth,depth_unit,water_sample_location,can_see_bottom,bank_material,bedrock_type,freshwater_habitat,salt_habitat,harvard_class


In [ ]:

# Second pass for rows previously labeled as 'Unknown'
unknown_indices = globe_site_data.index[globe_site_data['harvard_class'] == 'Unknown'].tolist()
second_pass_unknowns = []

# Column names
column_names = [col.replace('_', ' ').capitalize() + ':' for col in globe_site_data.columns if col != 'harvard_class']

# Optionally modify the prompt logic if needed
for count, idx in enumerate(unknown_indices, start=1):
    row = globe_site_data.loc[idx]

    # Create a more targeted prompt (optional)
    row_info = "\n".join(
        f"{col_name} {value}" for col_name, value in zip(column_names, row[globe_site_data.columns])
    )
    #row_info = f"The following record was previously unclassifiable. Try harder to classify it:\n{row_info}"

    # Re-run LLM classification
    llm_class_response, output_text = llm_labeler(row_info)

    # Track results
    if llm_class_response == 'Unknown':
        second_pass_unknowns.append(output_text)

    # Update DataFrame
    globe_site_data.at[idx, 'harvard_class'] = llm_class_response

    # Checkpoint save in data_path
    if count % 100 == 0:
        globe_site_data.to_csv(data_path + "globe_site_data_second_pass.csv", index=False)
        print(f"Second pass: processed {count} rows. Remaining 'Unknowns': {len(globe_site_data[globe_site_data['harvard_class'] == 'Unknown'])}\n")
        # Print the last 5 result from the unknown list
        new_five_exmaples = second_pass_unknowns[-5:]
        for example in new_five_exmaples:
            print(example)
        print("\n\n\n")

# Final save
globe_site_data.to_csv(data_path + "globe_site_data_20250227.csv", index=False)
print("Second pass complete. Final file saved.")


Second pass complete. Final file saved.
